In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['SN']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('substantia nigra', 822),
 ('salience network', 181),
 ('supernatant', 111),
 ('secretoneurin', 36),
 ('substantia nigra pars compacta', 29),
 ('sentinel node', 26),
 ('sciatic nerve', 23),
 ('streptonigrin', 20),
 ('salience', 17),
 ('sensory neurons', 15),
 ('sinomenine', 12),
 ('sinus node', 12),
 ('serum neutralizing', 11),
 ('sensitivity', 10),
 ('silver nanoparticles', 9),
 ('spontaneous nystagmus', 9),
 ('sodium nitrite', 9),
 ('selective negativity', 8),
 ('single nucleotide', 8),
 ('superficial neuromasts', 8),
 ('staphylococcal nuclease', 8),
 ('silver nitrate', 7),
 ('sodium nitroprusside', 6),
 ('saddle node', 6),
 ('succinonitrile', 6),
 ('single layer nanoemulsion', 6),
 ('social network', 5),
 ('splanchnic nerve', 5),
 ('spinal nerve', 5),
 ('sensory nerve', 5),
 ('sentinel lymph node', 5),
 ('solanum nigrum', 4),
 ('sufficient nitrogen', 4),
 ('substantial nigra', 4),
 ('sympathetic nerve', 4),
 ('staphylococcus aureus nuclease', 4),
 ('seed number per pod', 4),
 ('no

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-23 03:29:01] indra.ontology.bio.ontology - Initializing INDRA bio ontology for the first time, this may take a few minutes...
INFO: [2020-10-23 03:29:01] indra.ontology.bio.ontology - Adding nodes...
INFO: [2020-10-23 03:29:52] indra.ontology.bio.ontology - Adding xrefs...
INFO: [2020-10-23 03:30:07] indra.ontology.bio.ontology - Adding hierarchy...
INFO: [2020-10-23 03:30:34] indra.ontology.bio.ontology - Building name lookup...
INFO: [2020-10-23 03:30:39] indra.ontology.bio.ontology - Finished initializing bio ontology...
INFO: [2020-10-23 03:30:39] indra.ontology.bio.ontology - Caching INDRA bio ontology at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


KeyboardInterrupt: 

In [ ]:
result = [grounding_map, names, pos_labels]

In [ ]:
result

In [7]:
grounding_map, names, pos_labels = [{'non polar stem extracts': 'ungrounded',
  'saddle node': 'ungrounded',
  'salience': 'salience_network',
  'salience network': 'salience_network',
  'sciatic nerve': 'MESH:D012584',
  'secretoneurin': 'MESH:C081159',
  'seed number per pod': 'ungrounded',
  'selective negativity': 'ungrounded',
  'self namely': 'ungrounded',
  'sensitivity': 'ungrounded',
  'sensory nerve': 'ungrounded',
  'sensory neurons': 'MESH:D011984',
  'sensory neuropathy': 'NCIT:C3501',
  'sentinel lymph node': 'MESH:D000071036',
  'sentinel node': 'MESH:D000071036',
  'serum neutralizing': 'ungrounded',
  'silver nanoparticles': 'CHEBI:CHEBI:50826',
  'silver nitrate': 'CHEBI:CHEBI:32130',
  'single layer nanoemulsion': 'ungrounded',
  'single nucleotide': 'ungrounded',
  'sinomenine': 'CHEBI:CHEBI:9163',
  'sinus node': 'MESH:D012849',
  'social network': 'ungrounded',
  'sodium nitrite': 'CHEBI:CHEBI:78870',
  'sodium nitroprusside': 'CHEBI:CHEBI:29321',
  'sodium nucleinate': 'MESH:C013683',
  'solanum nigrum': 'MESH:D032324',
  'spikelet number': 'ungrounded',
  'spinal nerve': 'MESH:D013127',
  'spirogyra neglecta': 'ungrounded',
  'splanchnic nerve': 'MESH:D013153',
  'spontaneous nystagmus': 'MESH:D009759',
  'sqstm1 nup214': 'HGNC:8064',
  'standard nest': 'ungrounded',
  'staphylococcal nuclease': 'MESH:D008836',
  'staphylococcus aureus nuclease': 'MESH:D008836',
  'streptonigrin': 'CHEBI:CHEBI:9287',
  'subsequent neoplasms': 'MESH:D009369',
  'substantia nigra': 'MESH:D013378',
  'substantia nigra pars compacta': 'MESH:D013378',
  'substantia nigral': 'MESH:D013378',
  'substantial nigra': 'MESH:D013378',
  'succinonitrile': 'MESH:C010337',
  'sudomotor nerve': 'ungrounded',
  'sufficient nitrogen': 'CHEBI:CHEBI:17997',
  'superficial neuromasts': 'ungrounded',
  'supernatant': 'supernatant',
  'sympathetic nerve': 'ungrounded'},
 {'salience_network': 'salience_network',
  'MESH:D012584': 'Sciatic Nerve',
  'MESH:C081159': 'secretoneurin',
  'MESH:D011984': 'Sensory Receptor Cells',
  'NCIT:C3501': 'Peripheral Sensory Neuropathy',
  'MESH:D000071036': 'Sentinel Lymph Node',
  'CHEBI:CHEBI:50826': 'silver nanoparticle',
  'CHEBI:CHEBI:32130': 'silver(1+) nitrate',
  'CHEBI:CHEBI:9163': 'Sinomenine',
  'MESH:D012849': 'Sinoatrial Node',
  'CHEBI:CHEBI:78870': 'sodium nitrite',
  'CHEBI:CHEBI:29321': 'sodium nitroprusside',
  'MESH:C013683': 'sodium nucleinate',
  'MESH:D032324': 'Solanum nigrum',
  'MESH:D013127': 'Spinal Nerves',
  'MESH:D013153': 'Splanchnic Nerves',
  'MESH:D009759': 'Nystagmus, Pathologic',
  'HGNC:8064': 'NUP214',
  'MESH:D008836': 'Staphylococcal nuclease',
  'CHEBI:CHEBI:9287': 'streptonigrin',
  'MESH:D009369': 'Neoplasms',
  'MESH:D013378': 'Substantia Nigra',
  'MESH:C010337': 'succinonitrile',
  'CHEBI:CHEBI:17997': 'dinitrogen',
  'supernatant': 'supernatant'},
['CHEBI:CHEBI:9287',
  'MESH:C081159',
  'MESH:D009759',
  'MESH:D011984',
  'MESH:D012849',
  'MESH:D013378',
  'MESH:D000071036']]

In [8]:
excluded_longforms = []

In [9]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [11]:
additional_entities = {}

In [12]:
unambiguous_agent_texts = {}

In [13]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [ ]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [ ]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [17]:
intersection1

[('HGNC:17981', 'HGNC:17981', 29)]

In [18]:
intersection2

[('ungrounded', 'HGNC:17981', 0)]

In [20]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=['RTCA', 'RTCD1', 'RPC', 'RTC1', 'RTC'])
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [ ]:
names.update(additional_entitie)

In [14]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-23 03:40:41] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-23 03:41:16] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9268078878976048 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [17]:
classifier.stats

{'label_distribution': {'MESH:D013378': 599,
  'CHEBI:CHEBI:78870': 7,
  'CHEBI:CHEBI:29321': 5,
  'supernatant': 71,
  'CHEBI:CHEBI:9287': 14,
  'MESH:D009759': 9,
  'MESH:C013683': 2,
  'MESH:D012849': 11,
  'MESH:C081159': 29,
  'ungrounded': 60,
  'MESH:D012584': 18,
  'MESH:D011984': 10,
  'MESH:D000071036': 20,
  'CHEBI:CHEBI:9163': 9,
  'CHEBI:CHEBI:32130': 6,
  'MESH:D013153': 4,
  'IP:SSF50199': 9,
  'NCIT:C3501': 2,
  'CHEBI:CHEBI:50826': 5,
  'MESH:D013127': 4,
  'MESH:D032324': 3,
  'salience_network': 102,
  'MESH:C010337': 4,
  'CHEBI:CHEBI:17997': 2,
  'MESH:D009369': 2,
  'HGNC:8064': 1},
 'f1': {'mean': 0.926808, 'std': 0.011128},
 'precision': {'mean': 0.903974, 'std': 0.01702},
 'recall': {'mean': 0.960995, 'std': 0.013389},
 'ungrounded': {'f1': {'mean': 0.769258, 'std': 0.078383},
  'pr': {'mean': 0.7, 'std': 0.1},
  'rc': {'mean': 0.870303, 'std': 0.115524}},
 'CHEBI:CHEBI:9287': {'f1': {'mean': 0.644762, 'std': 0.173522},
  'pr': {'mean': 0.566667, 'std': 0.24944

In [15]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [16]:
disamb.dump(model_name, results_path)

In [17]:
print(disamb.info())

Disambiguation model for SN

Produces the disambiguations:
	NUP214	HGNC:8064
	Neoplasms	MESH:D009369
	Nystagmus, Pathologic*	MESH:D009759
	Peripheral Sensory Neuropathy	NCIT:C3501
	Sciatic Nerve	MESH:D012584
	Sensory Receptor Cells*	MESH:D011984
	Sentinel Lymph Node*	MESH:D000071036
	Sinoatrial Node*	MESH:D012849
	Sinomenine	CHEBI:CHEBI:9163
	Solanum nigrum	MESH:D032324
	Spinal Nerves	MESH:D013127
	Splanchnic Nerves	MESH:D013153
	Staphylococcal nuclease	MESH:D008836
	Substantia Nigra*	MESH:D013378
	dinitrogen	CHEBI:CHEBI:17997
	salience_network	salience_network
	secretoneurin*	MESH:C081159
	silver nanoparticle	CHEBI:CHEBI:50826
	silver(1+) nitrate	CHEBI:CHEBI:32130
	sodium nitrite	CHEBI:CHEBI:78870
	sodium nitroprusside	CHEBI:CHEBI:29321
	sodium nucleinate	MESH:C013683
	streptonigrin*	CHEBI:CHEBI:9287
	succinonitrile	MESH:C010337
	supernatant	supernatant

Class level metrics:
--------------------
Grounding                    	Count	F1     
             Substantia Nigra*	599	0.94691
   

In [18]:
model_to_s3(disamb)